# 02 - preprocesado

---



En este cuaderno realizamos la **Carga y preprocesado de Datos** del conjunto de entrenamiento de la competición de Kaggle, siguiendo los requisitos de la entrega 2 del curso.


### **descargamos los datos de la competencia en kaggle**



In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/kaggle'
!chmod 600 /kaggle/kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia

udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip: Skipping, found more recently modified local copy (use --force to force download)


## descomprimimos y revisamos los archivos de la competencia

In [2]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [3]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185238 test.csv
   692501  10666231 143732437 train.csv
  1286075  15528571 207634348 total


## cargamos el `train.csv` con pandas



In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Cargar datos de entrenamiento
z = pd.read_csv('train.csv')

# Verificar tamaño del DataFrame
print('Tamaño del DataFrame:', z.shape)




Tamaño del DataFrame: (692500, 21)


In [5]:

# 1. librerías

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline


# 2. Carga de los datos

# Ajusta la ruta según tu entorno
train = z
print("Shape del dataset:", train.shape)
train.head()





Shape del dataset: (692500, 21)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [6]:
# 3. Inspección inicial

print("\nColumnas con valores faltantes:\n")
print(train.isnull().sum()[train.isnull().sum() > 0])

print("\nClases en la variable objetivo:")
print(train["RENDIMIENTO_GLOBAL"].value_counts())





Columnas con valores faltantes:

E_VALORMATRICULAUNIVERSIDAD     6287
E_HORASSEMANATRABAJA           30857
F_ESTRATOVIVIENDA              32137
F_TIENEINTERNET                26629
F_EDUCACIONPADRE               23178
F_TIENELAVADORA                39773
F_TIENEAUTOMOVIL               43623
E_PAGOMATRICULAPROPIO           6498
F_TIENECOMPUTADOR              38103
F_TIENEINTERNET.1              26629
F_EDUCACIONMADRE               23664
dtype: int64

Clases en la variable objetivo:
RENDIMIENTO_GLOBAL
alto          175619
bajo          172987
medio-bajo    172275
medio-alto    171619
Name: count, dtype: int64


In [7]:
# 4. Separar variables predictoras y variable objetivo

target = "RENDIMIENTO_GLOBAL"
X = train.drop(columns=[target, "ID"])  # quitamos el ID porque no aporta información
y = train[target]

# 5. Identificación de tipos de variables

# Variables numéricas (en este dataset, usualmente los INDICADORES son numéricos)
num_cols = [col for col in X.columns if col.startswith("INDICADOR_")]

# Variables categóricas (resto)
cat_cols = [col for col in X.columns if col not in num_cols]

print(f"\nVariables numéricas: {num_cols}")
print(f"\nVariables categóricas: {cat_cols[:10]} ... (total: {len(cat_cols)})")


Variables numéricas: ['INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4']

Variables categóricas: ['PERIODO_ACADEMICO', 'E_PRGM_ACADEMICO', 'E_PRGM_DEPARTAMENTO', 'E_VALORMATRICULAUNIVERSIDAD', 'E_HORASSEMANATRABAJA', 'F_ESTRATOVIVIENDA', 'F_TIENEINTERNET', 'F_EDUCACIONPADRE', 'F_TIENELAVADORA', 'F_TIENEAUTOMOVIL'] ... (total: 15)


In [8]:
# 6. Imputación de valores faltantes

# - numéricas: media
# - categóricas: valor más frecuente (moda)

num_imputer = SimpleImputer(strategy="mean")
cat_imputer = SimpleImputer(strategy="most_frequent")


# 7. Codificación de variables categóricas

# Usamos OneHotEncoder para variables categóricas
# handle_unknown='ignore' evita errores si aparecen categorías nuevas

cat_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)


# 8. Escalamiento de variables numéricas

scaler = StandardScaler()


# 9. Creación del preprocesador general

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', num_imputer),
            ('scaler', scaler)
        ]), num_cols),

        ('cat', Pipeline(steps=[
            ('imputer', cat_imputer),
            ('encoder', cat_encoder)
        ]), cat_cols)
    ]
)




In [9]:
# 10. Aplicación del preprocesamiento

X_preprocessed = preprocessor.fit_transform(X)


# 11. Codificación de la variable objetivo

# Convertimos las etiquetas a números (bajo=0, medio-bajo=1, medio-alto=2, alto=3)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


print(f"\nNaNs\n")
pd.DataFrame(X_preprocessed).isnull().sum().sum()


NaNs



np.int64(0)

In [10]:
# 12. Verificación final

print("Forma final del dataset preprocesado:", X_preprocessed.shape)
print("Ejemplo de etiquetas codificadas:", np.unique(y_encoded))



Forma final del dataset preprocesado: (692500, 1050)
Ejemplo de etiquetas codificadas: [0 1 2 3]


In [11]:

print("\n      Preprocesamiento completado correctamente.")
print(f"X_preprocessed.npy: {X_preprocessed}")
print(f"y_encoded.npy: {y_encoded}")

print("\n\nVariables finales:", X_preprocessed.shape[1])
print("Observaciones:", X_preprocessed.shape[0])


      Preprocesamiento completado correctamente.
X_preprocessed.npy: [[ 0.43700203 -0.55622304  0.81397806 ...  0.          0.
   0.        ]
 [ 0.34693377 -0.48134083  0.50818004 ...  0.          0.
   1.        ]
 [ 0.23230144 -0.49203829  0.72903416 ...  0.          0.
   0.        ]
 ...
 [ 0.14223317 -0.21390437  0.88193317 ...  1.          0.
   0.        ]
 [-1.1187225   1.7758229  -0.01847211 ...  0.          0.
   0.        ]
 [ 1.78802596 -0.85575188 -0.1883599  ...  0.          1.
   0.        ]]
y_encoded.npy: [2 1 1 ... 3 1 0]


Variables finales: 1050
Observaciones: 692500


### Interpretación del preprocesamiento



- Durante esta fase se realizó la limpieza y transformación de los datos con el objetivo de obtener un conjunto de entrenamiento consistente y adecuado para la etapa de modelado.
En particular:

- Se eliminaron columnas irrelevantes para la predicción, como el identificador ID.

- Se detectaron y trataron valores faltantes, imputando la media para variables numéricas y la moda para categóricas, con el fin de no perder observaciones valiosas.

- Las variables categóricas fueron codificadas mediante OneHotEncoder, garantizando que el modelo pueda interpretar las categorías sin introducir relaciones numéricas falsas.

- Los indicadores cuantitativos (INDICADOR_1, INDICADOR_2, INDICADOR_3, INDICADOR_4) se escalaron con StandardScaler, homogeneizando su rango de variación.

- La variable objetivo RENDIMIENTO_GLOBAL fue etiquetada numéricamente para permitir su uso en modelos supervisados de clasificación multiclase.

- Con estas transformaciones, el dataset resultante (X_preprocessed) quedó completamente numérico, sin valores nulos, normalizado y balanceado en estructura, lo cual facilita el entrenamiento de algoritmos como Random Forest, Logistic Regression o XGBoost en la siguiente entrega.


## Conclusiones del preprocesamiento

- El conjunto de datos fue preparado correctamente para modelado, manteniendo la integridad y la variabilidad de las observaciones. El preprocesamiento logró estandarizar y limpiar la base de datos sin pérdida de información relevante.


- Se redujo el riesgo de sesgos derivados de categorías con valores ausentes o escalas dispares.

- Las decisiones tomadas (imputación, normalización, codificación) siguen buenas prácticas recomendadas en tareas de clasificación multiclase.

- Esta fase sienta las bases para aplicar modelos supervisados y evaluar su desempeño mediante métricas como accuracy, coherente con la métrica de evaluación de la competencia Kaggle.

- En comparación con el ejemplo completo del profesor, esta entrega se limita al alcance solicitado: preprocesamiento y limpieza. No incluye aún selección de características ni modelado predictivo, ya que esas etapas corresponden a las entregas posteriores.